# Augmentation
Create synthetic data by SMOTE method

## Import libraries

In [1]:
# Add parent directory to system path in order to import custom modules
import config as conf
import numpy as np
import h5py
from batchcreator import load_fns_pysteps
from matplotlib import pyplot as plt
from pysteps.visualization import plot_precip_field

from pysteps.io import archive
from datetime import datetime, timedelta


Pysteps configuration file found at: /Users/mozykhau/.conda/envs/precipitation-nowcasting-using-GANs/lib/python3.7/site-packages/pysteps/pystepsrc



In [2]:
def eventGeneration(start_time, obs_time = 4 ,lead_time = 72):
    # Generate event based on starting time point, return a list: [[t-4,...,t-1,t], [t+1,...,t+72]]
    # Get the start year, month, day, hour, minute
    year = int(start_time[0:4])
    month = int(start_time[4:6])
    day = int(start_time[6:8])
    hour = int(start_time[8:10])
    minute = int(start_time[10:12])
    #print(datetime(year=year, month=month, day=day, hour=hour, minute=minute))
    times = [(datetime(year, month, day, hour, minute) + timedelta(minutes=5 * (x+1))) for x in range(lead_time)]
    lead = [dt.strftime('%Y%m%d%H%M') for dt in times]
    times = [(datetime(year, month, day, hour, minute) - timedelta(minutes=5 * x)) for x in range(obs_time)]
    obs = [dt.strftime('%Y%m%d%H%M') for dt in times]
    obs.reverse()
    return lead, obs

## SMOTE Augmentation

Download extreme dataset

In [9]:
file = '/Users/mozykhau/University/Extreme event project/Data/Extreme events/Delfland extremes/extreme datesDelfland_2008-2021_sum.npy'
extreme_events = np.load(file)
length = extreme_events.shape
print("Total number of extreme events", length[0])

Total number of extreme events 2176


In [11]:
extreme_events[:5]

array([202106192210, 202106192205, 202106192200, 202106192215,
       202106192155])

In [ ]:
from random import uniform
list_of_events = extreme_events
# list_of_events = list_of_events[:10]
#  Parameters for the loading of files
root_path = conf.dir_rtcor    # directions to data directory
path_fmt = "%Y/%m" # how files are sorted in folders (no subfolders)
fn_pattern = conf.prefix_rtcor+"%Y%m%d%H%M" # filename pattern
fn_ext = "h5"   # filename extension
timestep = 30
t =  36 # lead time
importer_name = "knmi_hdf5"
importer_kwargs = {"accutime": 5,
                  "qty": "ACRR",
                  "pixelsize": 1.0}                       # dict of importer arguments for qty, accutime, pixelsize
prep_average = 0
peak_average = 0
overview = {}
new_event = {}
for i in range(list_of_events.__len__()):
    event = list_of_events[i]
    lead_times, obs_times = eventGeneration(event, obs_time=5, lead_time=t)
    sample_times = obs_times + lead_times
    neighbour_int_average = np.zeros([sample_times.__len__(),765,700])
    count = 0
    for u in range(-3,3):
        if i+u<0 or u==0 or i+u>list_of_events.__len__()-1:
            continue
        neighbour_event = list_of_events[i+u]
        lead_times, obs_times = eventGeneration(neighbour_event, obs_time=5, lead_time=t)
        sample_times_neighbour = obs_times + lead_times
        for times_nb in sample_times_neighbour:
            dt_inp = datetime.strptime(times_nb, "%Y%m%d%H%M")
            filename = archive.find_by_date(dt_inp, root_path, path_fmt, fn_pattern, fn_ext, timestep=timestep)
            address = filename[0]
            f = h5py.File(address[0])['image1']['image_data']
            f = np.array(f)
            # f = np.where(f == 65535, -1, f)
            # f = np.ma.masked_where(f <= 0, f)
            f[f == 65535] = 0
            # f = f[100:500, 100:500] * 12 / 100
            f = f * 12 / 100
            neighbour_int_average[sample_times_neighbour.index(times_nb),:,:] = np.add(neighbour_int_average[sample_times_neighbour.index(times_nb),:,:],f)
        count += 1
    neighbour_int_average = neighbour_int_average/count

    #Modify initial event sequence
    for times in sample_times:
        dt_inp = datetime.strptime(times, "%Y%m%d%H%M")
        filename = archive.find_by_date(dt_inp, root_path, path_fmt, fn_pattern, fn_ext, timestep=timestep)
        address = filename[0]
        f = h5py.File(address[0])['image1']['image_data']
        f = np.array(f)
        f[f == 65535] = 0
        f = f * 12 / 100
        new_event[times] = np.add(f,uniform(0,1)*np.subtract(f,neighbour_int_average[sample_times.index(times),:,:]))

In [ ]:
list(new_event.keys())

## Visualise results

In [ ]:
index_event = '200806021900'
val_row = [[index_event],[index_event]]
_,_,metadata,_ = load_fns_pysteps(val_row)
lead_times, obs_times = eventGeneration(index_event, obs_time=5, lead_time=t)
subplot_number = obs_times.__len__()
# Visaulise maps before event
plt.rc('font', size=8)
plt.figure(0,figsize=(20, 8), dpi=100)
for time in obs_times:
    dt_inp = datetime.strptime(time, "%Y%m%d%H%M")
    filename = archive.find_by_date(dt_inp, root_path, path_fmt, fn_pattern, fn_ext, timestep=timestep)
    address = filename[0]
    f = h5py.File(address[0])['image1']['image_data']
    f = np.array(f)
    f[f == 65535] = 0
    f = f * 12 / 100
    plt.subplot(2,subplot_number,obs_times.index(time)+1)
    plot_precip_field(f,geodata=metadata, axis="on")
    plt.title('Real prep ' + time)
    plt.subplot(2,subplot_number,obs_times.index(time)+subplot_number+1)
    plot_precip_field(new_event[time],geodata=metadata, axis="on")
    plt.title('Augmented prep ' + time)


subplot_number_2 = lead_times[::6].__len__()
plt.figure(1,figsize=(20, 8), dpi=100)
lead_times_for_plot = lead_times[::6]
for time in lead_times_for_plot:
    dt_inp = datetime.strptime(time, "%Y%m%d%H%M")
    filename = archive.find_by_date(dt_inp, root_path, path_fmt, fn_pattern, fn_ext, timestep=timestep)
    address = filename[0]
    f = h5py.File(address[0])['image1']['image_data']
    f = np.array(f)
    f[f == 65535] = 0
    f = f * 12 / 100
    plt.subplot(2,subplot_number_2,lead_times_for_plot.index(time)+1)
    plot_precip_field(f,geodata=metadata, axis="on")
    plt.title('Real prep ' + time)
    plt.subplot(2,subplot_number_2,lead_times_for_plot.index(time)+subplot_number_2+1)
    plot_precip_field(new_event[time],geodata=metadata, axis="on")
    plt.title('Augmented ' + time)